In [1]:
from neural_astar_jax.utils import data
from neural_astar_jax.utils import training
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from importlib import reload
from neural_astar_jax.planner import astar, differentiable_astar
reload(data)
reload(training)
reload(differentiable_astar)
reload(astar)
import optax
from flax.training.train_state import TrainState

In [2]:
planner = astar.NeuralAstar(is_training=True, search_step_ratio=0.25)
train_loader = data.MazeDataLoader(filename="../../planning-datasets/data/mpd/mazes_032_moore_c8.npz", split="train", batch_size=100)
val_loader = data.MazeDataLoader(filename="../../planning-datasets/data/mpd/mazes_032_moore_c8.npz", split="val", batch_size=1)
trainer = training.Trainer(planner=planner, train_loader=train_loader, val_loader=val_loader, val_every_n_steps=1)

In [4]:
state = trainer.fit(jax.random.PRNGKey(0), max_steps=10)

step=00, train_loss=0.0623, val_loss=0.1113, p_opt=0.5700, p_exp=0.2878, h_mean=0.3825
best model updated (h_mean=0.3825 > best_h_mean=-inf)
step=01, train_loss=0.0836, val_loss=0.1106, p_opt=0.5900, p_exp=0.2892, h_mean=0.3882
best model updated (h_mean=0.3882 > best_h_mean=0.3825)
step=02, train_loss=0.0666, val_loss=0.1106, p_opt=0.5800, p_exp=0.2900, h_mean=0.3867
step=03, train_loss=0.0664, val_loss=0.1092, p_opt=0.6000, p_exp=0.2976, h_mean=0.3978
best model updated (h_mean=0.3978 > best_h_mean=0.3882)
step=04, train_loss=0.0739, val_loss=0.1057, p_opt=0.6000, p_exp=0.3202, h_mean=0.4175
best model updated (h_mean=0.4175 > best_h_mean=0.3978)
step=05, train_loss=0.0655, val_loss=0.1052, p_opt=0.5600, p_exp=0.3253, h_mean=0.4115
step=06, train_loss=0.0759, val_loss=0.1048, p_opt=0.5400, p_exp=0.3259, h_mean=0.4065
step=07, train_loss=0.0641, val_loss=0.1042, p_opt=0.5500, p_exp=0.3280, h_mean=0.4109
step=08, train_loss=0.0750, val_loss=0.1019, p_opt=0.5900, p_exp=0.3388, h_mean=0.